# 安装步骤

In [ ]:
from lib2to3.pgen2.grammar import line

linux控制台上使用root安装登录jupyter


# 更新apt源，输入你的sudo密码
sudo apt update && sudo apt upgrade -y

# 安装Python3、pip3和必要依赖（Ubuntu默认没有预装pip）
sudo apt install -y python3 python3-pip python3-dev

# 升级pip3，加--user避免权限问题
pip3 install --upgrade pip

# 安装JupyterLab
pip3 install jupyterlab
# （可选）如果需要支持扩展插件，安装扩展管理器
pip3 install jupyterlab_widgets

jupyter lab --version

# 1. 获取 jupyter 可执行文件路径
which jupyter
# 输出示例：/home/你的用户名/.local/bin/jupyter

#2.创建并编辑服务文件：
sudo vim /etc/systemd/system/jupyter-lab.service
<<<
[Unit]
Description=JupyterLab Service (Root User)
After=network.target

[Service]
Type=simple
User=root
ExecStart=/usr/local/bin/jupyter lab --ip=0.0.0.0 --port=8888 --no-browser --allow-root
WorkingDirectory=/home/wanglei/pyenv
Restart=always
RestartSec=10
StandardOutput=append:/var/log/jupyter-lab.log
StandardError=append:/var/log/jupyter-lab.err.log
LimitNOFILE=65535

[Install]
WantedBy=multi-user.target
>>>

# 重载systemd配置（让修改生效）
sudo systemctl daemon-reload

# 重启服务
sudo systemctl restart jupyter-lab.service

# 查看服务状态（此时应显示active (running)）
sudo systemctl status jupyter-lab.service

# 检查8888端口
sudo netstat -tlnp | grep 8888

#执行密码设置命令
# 用root权限执行（和服务运行用户一致）
sudo jupyter server password

In [ ]:
#一、全量卸载（系统级 + Python 包，彻底清空）
#1. 卸载系统级 CUDA Toolkit（若已安装，全量清理）
#适用于 Linux 系统（Ubuntu/CentOS 通用），彻底卸载所有 CUDA 版本：

# 停止CUDA相关服务（如有）
sudo systemctl stop nvidia-persistenced 2>/dev/null

# 卸载CUDA Toolkit（覆盖12.x所有版本）
sudo apt-get purge -y nvidia* cuda* cudnn* 2>/dev/null
sudo apt-get autoremove -y
sudo rm -rf /usr/local/cuda* /usr/lib/cuda* /usr/include/cuda*
sudo rm -rf ~/.nv ~/.cuda

# 清理CUDA环境变量（避免残留）
sed -i '/CUDA_HOME/d' ~/.bashrc
sed -i '/cuda/d' ~/.bashrc
sed -i '/NVIDIA/d' ~/.bashrc
source ~/.bashrc

#2. 卸载所有 Python 依赖（全量清理）
pip3 uninstall -y \
    torch torchvision torchaudio \
    transformers auto-gptq optimum peft \
    modelscope pyarrow numpy bitsandbytes triton datasets \
    accelerate sentencepiece safetensors

# 清理pip缓存+残留文件
pip3 cache purge
rm -rf ~/.cache/huggingface ~/.cache/modelscope

In [ ]:
#步骤 1：安装适配的 CUDA Toolkit 12.1
#####cuda版本需注意安装12.1版本，按照你的显卡、cuda实际版本参考，尽量选择稳定版本
nvidia-smi #查看当前硬件版本和cuda驱动
nvcc -V  # 输出CUDA Version 12.1即正确

# 因我的环境是cuda是12.8，在选择量化时报错bitsandbytes 预编译包、triton 依赖不兼容，所以需要卸载CUDA12.8，换成12.1
# 1. 安装CUDA 12.1依赖
sudo apt-get update && sudo apt-get install -y \
    gcc g++ make linux-headers-$(uname -r)

# 2. 下载并安装CUDA 12.1（Ubuntu 20.04/22.04通用）
wget https://developer.download.nvidia.com/compute/cuda/12.1.0/local_installers/cuda_12.1.0_530.30.02_linux.run
sudo sh cuda_12.1.0_530.30.02_linux.run --silent --toolkit
rm -f cuda_12.1.0_530.30.02_linux.run

# 3. 配置CUDA环境变量（生效当前会话+永久生效）
echo "export CUDA_HOME=/usr/local/cuda-12.1" >> ~/.bashrc
echo "export PATH=\$CUDA_HOME/bin:\$PATH" >> ~/.bashrc
echo "export LD_LIBRARY_PATH=\$CUDA_HOME/lib64:\$LD_LIBRARY_PATH" >> ~/.bashrc
source ~/.bashrc

# 验证CUDA安装
nvcc -V  # 应输出 CUDA Version 12.1.0


In [ ]:
#步骤 2：安装 PyTorch（基础依赖，先装核心框架）
#严格按官网量化要求选torch==2.1，且匹配 CUDA 12.1
#####pytorch则需要按照cuda的版本进行选择安装，可参考官网 https://pytorch.org/get-started/locally/
# CUDA 12.1
pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121

#此处安装可能会有警告，可以跳过

In [ ]:
#步骤 3：安装量化相关依赖
# 1. 先装基础依赖（numpy/pyarrow，避免量化包安装时依赖缺失）
pip3 install numpy==1.26.4 pyarrow==14.0.1 sentencepiece safetensors

# 2. 装官网要求的量化核心包（严格匹配版本）
pip3 install \
transformers==4.34.1 \
auto-gptq==0.5.1 \
optimum==1.14.0 \
peft==0.6.2 \
modelscope==1.14.0 \
accelerate==0.24.1 \
huggingface_hub==0.19.4 \
deepspeed==0.14.0
openmpi-bin openmpi-common libopenmpi-dev  mpi4py


In [ ]:
#linux控制台上qwen1.8B 安装
#参考地址：https://github.com/QwenLM/Qwen/blob/main/README_CN.md

# 1. 安装git（如果未安装）
sudo apt install -y git

# 2. 克隆Qwen官方仓库（包含requirements.txt）
cd /home/wanglei/pyenv
git clone https://github.com/QwenLM/Qwen.git
cd Qwen/

#优选：安装transformers指定版本：4.32.0 （需配套）
/usr/bin/pip3 install transformers=4.32.0
    
#安装版本其他依赖
pip3 install -r requirements.txt

# 检查步骤 python代码

In [3]:
# 1. 检查Python版本
!python3 --version

Python 3.10.12


In [4]:
# 2. 检查Jupyter内核Python版本
import sys
print(f"Jupyter内核Python版本：{sys.version.split()[0]}")

Jupyter内核Python版本：3.10.12


In [2]:
# 3. 检查PyTorch版本
import torch
print(f"PyTorch版本：{torch.__version__}")

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:51: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


PyTorch版本：2.1.2+cu121


In [5]:
# 4. 检查Transformers版本
import transformers
print(f"Transformers版本：{transformers.__version__}")

Transformers版本：4.34.1


In [6]:
# 5. 检查CUDA是否可用
import torch
print(f"CUDA是否可用：{torch.cuda.is_available()}")
print(f"CUDA版本（PyTorch检测）：{torch.version.cuda if torch.cuda.is_available() else '未安装CUDA'}")

CUDA是否可用：True
CUDA版本（PyTorch检测）：12.1


In [7]:
# 6. 检查系统CUDA版本
!nvcc -V  # 查看已安装的CUDA Toolkit版本
!nvidia-smi  # 查看显卡驱动支持的最高CUDA版本

/bin/sh: 1: nvcc: not found
Wed Feb  4 09:38:28 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.18             Driver Version: 570.172.18     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX5880-Ada-8Q          Off |   00000000:00:09.0 Off |                  N/A |
| N/A   N/A    P0            N/A  /  N/A  |    2387MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------

# 加载 Qwen-1.8B-Chat-Int4

In [8]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. 下载量化模型
model_dir = snapshot_download(
    'qwen/Qwen-1_8B-Chat-Int4',
    revision='v1.0.0',
    cache_dir='../qwen_model'
)

# 2. 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

# 3. 关键：AutoModelForCausalLM加载量化模型（依赖AutoGPTQ）
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

# 4. 测试推理
response, history = model.chat(tokenizer, "你好，请介绍一下自己", history=None)
print(f"✅ 模型回复：{response}")
print(f"✅ 模型运行设备：{next(model.parameters()).device}")  # 输出cuda:0即GPU生效


2026-02-04 09:38:42,058 - modelscope - INFO - PyTorch version 2.1.2+cu121 Found.
2026-02-04 09:38:42,059 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2026-02-04 09:38:42,089 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 dc2d27292ce4d5b00f912562a91fbfec and a total number of 976 components indexed
2026-02-04 09:38:42,877 - modelscope - INFO - Use user-specified model revision: v1.0.0
Try importing flash-attention for faster inference...


✅ 模型回复：您好！我是来自阿里云的大规模语言模型，我叫通义千问。作为阿里云的一员，我的主要功能是为用户提供自然语言处理服务。我可以回答问题、创作文字，还能表达观点、撰写代码。如果您有任何关于人工智能或者自然语言处理的问题，欢迎随时向我提问哦！
✅ 模型运行设备：cuda:0


In [ ]:
# prompt project

In [8]:
import random
import json
import time

with open('./city.txt', 'r', encoding='utf-8') as f:
    city_list = f.readlines()
    city_list = [line.strip().split(' ')[1] for line in city_list]

Q='青岛4月6日下雨么?'

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''


Q_arr = []
A_arr = []

Q_list = [
    ('{city}{year}年{month}月{day}日的天气','%Y-%m-%d'),
    ('{city}{year}年{month}月{day}号的天气','%Y-%m-%d'),
    ('{city}{month}月{day}日的天气','%m-%d'),
    ('{city}{month}月{day}号的天气','%m-%d'),

    ('{year}年{month}月{day}日{city}的天气','%Y-%m-%d'),
    ('{year}年{month}月{day}号{city}的天气','%Y-%m-%d'),
    ('{month}月{day}日{city}的天气','%m-%d'),
    ('{month}月{day}号{city}的天气','%m-%d'),

    ('你们{year}年{month}月{day}日去{city}玩吗？','%Y-%m-%d'),
    ('你们{year}年{month}月{day}号去{city}玩么？','%Y-%m-%d'),
    ('你们{month}月{day}日去{city}玩吗？','%m-%d'),
    ('你们{month}月{day}号去{city}玩吗？','%m-%d'),
]

# 生成1000条训练样本（日期转换/天气相关）
for i in range(1000):
    Q_template = Q_list[random.randint(0, len(Q_list)-1)]
    city = city_list[random.randint(0, len(city_list)-1)]
    year = random.randint(1990, 2025)
    month = random.randint(1, 12)
    day = random.randint(1, 28)
    time_str = '{}-{}-{}'.format(year, month, day)
    date_field = time.strftime(Q_template[1], time.strptime(time_str, '%Y-%m-%d'))
    question = Q_template[0].format(city=city, year=year, month=month, day=day)  # 原始问题
    answer = json.dumps({'city': city, 'date': date_field}, ensure_ascii=False)  # 回答

    Q_arr.append(prompt_template % question)
    A_arr.append(answer)

# ========== 核心修改：生成conversations格式 ==========
train_data = []
for q, a in zip(Q_arr, A_arr):
    train_data.append({
        "conversations": [
            {"from": "user", "value": q},       # 用户问题
            {"from": "assistant", "value": a}   # 助手回答
        ]
    })

# ========== 写入JSON文件（UTF-8编码，适配finetune脚本） ==========
output_json_path = "train.json"  # 输出的训练文件路径
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)  # indent=2方便查看，不影响训练

print(f"训练数据生成完成！文件路径：{output_json_path}")
print(f"共生成 {len(train_data)} 条训练样本")
# 打印第一条样本示例，方便你核对格式
print("\n第一条样本示例：")
print(json.dumps(train_data[0], ensure_ascii=False, indent=2))

训练数据生成完成！文件路径：train.json
共生成 1000 条训练样本

第一条样本示例：
{
  "conversations": [
    {
      "from": "user",
      "value": "\n给定一句话：“12月12日镇宁布依族苗族自治县的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中的城市和日期共2个信息\n步骤2：根据城市和日期信息，生成JSON字符串，格式为{\"city\":城市,\"date\":日期}\n\n请问，这个JSON字符串是：\n"
    },
    {
      "from": "assistant",
      "value": "{\"city\": \"镇宁布依族苗族自治县\", \"date\": \"12-12\"}"
    }
  ]
}


# 微调模型，生成到output_qwen

使用qwen提供的finetune脚本进行lora微调
qlora支持量化微调
ds 全参微调
single 单卡

脚本执行：
Usage: bash finetune/finetune_qlora_ds.sh [-m MODEL_PATH] [-d DATA_PATH]

In [17]:
import torch
# 清空所有GPU的PyTorch显存缓存
torch.cuda.empty_cache()
!nvidia-smi

Wed Feb  4 09:49:04 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.18             Driver Version: 570.172.18     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX5880-Ada-8Q          Off |   00000000:00:09.0 Off |                  N/A |
| N/A   N/A    P0            N/A  /  N/A  |    4961MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
%cd /home/wanglei/pyenv/Qwen-main/
!pwd
!bash finetune/finetune_qlora_single_gpu.sh -m /home/wanglei/pyenv/qwen_model/qwen/Qwen-1_8B-Chat-Int4 -d /home/wanglei/pyenv/qwen3-1B-sft-learn/train.json

/home/wanglei/pyenv/Qwen-main
/home/wanglei/pyenv/Qwen-main


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:51: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
[2026-02-04 09:50:28,646] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pre

In [ ]:
# 加载SFT后的模型

In [24]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, GPTQConfig, AutoModelForCausalLM

# ========== 1. 定义GPTQ量化配置（匹配微调时的模型量化方式） ==========
gptq_config = GPTQConfig(
    bits=4,  # 你的模型是4bit量化
    disable_exllama=True,  # 适配旧版transformers，避免冲突
    trust_remote_code=True
)

# ========== 2. 加载Peft模型（兼容QuantLinear） ==========
# 方式1：直接加载（推荐，自动合并LoRA适配器）
model = AutoPeftModelForCausalLM.from_pretrained(
    '../Qwen-main/output_qwen',  # Peft模型目录
    device_map="auto",
    trust_remote_code=True,
    quantization_config=gptq_config,  # 关键：指定量化配置，识别QuantLinear
    low_cpu_mem_usage=True  # 减少CPU内存占用，适配量化模型
).eval()

model.generation_config.top_p=0 # 只选择概率最高的token

Q_list=['2020年4月16号三亚下雨么？','青岛3-15号天气预报','5月6号下雪么，城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么，大概是1月15号左右，我们3个人一起']
#for Q in Q_list:
#    prompt=prompt_template%(Q,)
#    A,hist=model.chat(tokenizer,prompt,history=None)
#    print('Q:%s\nA:%s\n'%(Q,A))

history=None
while True:
    Q = input("请输入：").strip()
    prompt=prompt_template%(Q,)
    A, history = model.chat(tokenizer, prompt, history=history)
    print(f"回答：{A}\n")


You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
Try importing flash-attention for faster inference...


Q:2020年4月16号三亚下雨么？
A:{"city": "三亚", "date": "2020-04-16"}

Q:青岛3-15号天气预报
A:{"city": "青岛", "date": "03-15"}

Q:5月6号下雪么，城市是威海
A:{"city": "威海", "date": "05-06"}

Q:青岛2023年12月30号有雾霾么?
A:{"city": "青岛", "date": "2023-12-30"}

Q:我打算6月1号去北京旅游，请问天气怎么样？
A:{"city": "北京", "date": "06-01"}

Q:你们打算1月3号坐哪一趟航班去上海？
A:{"city": "上海", "date": "01-03"}

Q:小明和小红是8月8号在上海结婚么?
A:{"city": "上海", "date": "08-08"}

Q:一起去东北看冰雕么，大概是1月15号左右，我们3个人一起
A:{"city": "东北", "date": "01-15"}

